# Algorithm: Handling Conditional Discrimination

 ## Step 1: Data cleaning and processing

In [54]:
import pandas as pd

data = pd.read_csv("compas-scores-two-years.csv")
df = data[['age', 'c_charge_degree', 'race', 'sex', 'priors_count', 'decile_score', 'is_recid', 'two_year_recid']].copy()

# Encode categorical variables using DataFrame column assignment to avoid DeprecationWarning
df['race'] = df['race'].map({'African-American': 0, 'Caucasian': 1})
df['sex'] = df['sex'].map({'Male': 1, 'Female': 0})
df['c_charge_degree'] = df['c_charge_degree'].map({'F': 0, 'M': 1})

df.dropna(inplace=True)
df.head()

,age,c_charge_degree,race,sex,priors_count,decile_score,is_recid,two_year_recid
1,34,0,0.0,1,0,3,1,1
2,24,0,0.0,1,4,4,1,1
3,23,0,0.0,1,1,8,0,0
6,41,0,1.0,1,14,6,1,1
8,39,1,1.0,0,0,1,0,0


## Step 2: Applying Handling Conditional Discrimination

In [55]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, ClassifierMixin
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score, brier_score_loss
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve
import time

In [56]:
class FairLogisticRegression(BaseEstimator, ClassifierMixin):
    def __init__(self, lambda_=0.1):
        self.lambda_ = lambda_  # Regularization parameter for the fairness constraint
    
    def fit(self, X, y, sensitive_features):
        n_features = X.shape[1]
        initial_weights = np.zeros(n_features + 1)  # Including the intercept
        
        # Objective function to minimize: Logistic loss + fairness constraint penalty
        def objective(weights):
            scores = np.dot(X, weights[1:]) + weights[0]
            predictions = 1 / (1 + np.exp(-scores))
            
            # Logistic loss
            logistic_loss = -np.mean(y * np.log(predictions) + (1 - y) * np.log(1 - predictions))
            
            # Fairness constraint: Difference in FPR between subgroups
            mask_sensitive = sensitive_features == 1
            mask_nonsensitive = ~mask_sensitive
            fpr_sensitive = np.mean((predictions > 0.5) & (y == 0) & mask_sensitive)
            fpr_nonsensitive = np.mean((predictions > 0.5) & (y == 0) & mask_nonsensitive)
            fairness_penalty = np.abs(fpr_sensitive - fpr_nonsensitive)
            
            return logistic_loss + self.lambda_ * fairness_penalty
        
        # Minimize the objective function
        result = minimize(objective, initial_weights)
        self.weights_ = result.x
    
    def predict_proba(self, X):
        scores = np.dot(X, self.weights_[1:]) + self.weights_[0]
        predictions = 1 / (1 + np.exp(-scores))
        return np.vstack([1-predictions, predictions]).T
    
    def predict(self, X):
        return (self.predict_proba(X)[:, 1] > 0.5).astype(int)
    
    
# Train the model with fairness consideration
start_time = time.time()
fair_lr = FairLogisticRegression(lambda_=0.1)
fair_lr.fit(X_train.values, y_train.values, sensitive_features=X_train['race'].values)
training_time = time.time() - start_time

# Predict and evaluate
y_pred = fair_lr.predict(X_test.values)
y_proba = fair_lr.predict_proba(X_test.values)
print(f"Training Time: {training_time:} seconds")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Training Time: 2.2975988388061523 seconds
Accuracy: 0.5300813008130081


### Step 3: Evaluation

In [58]:
def evaluate_fairness(y_true, y_pred, sensitive_features, y_proba):
    # Separate the data into subgroups based on the sensitive attribute
    subgroup_1_mask = sensitive_features == 0
    subgroup_2_mask = sensitive_features == 1

    # Calculate accuracy for each subgroup
    accuracy_subgroup_1 = accuracy_score(y_true[subgroup_1_mask], y_pred[subgroup_1_mask])
    accuracy_subgroup_2 = accuracy_score(y_true[subgroup_2_mask], y_pred[subgroup_2_mask])

    # Calculate FPR for each subgroup
    fpr_subgroup_1 = np.mean((y_pred[subgroup_1_mask] == 1) & (y_true[subgroup_1_mask] == 0))
    fpr_subgroup_2 = np.mean((y_pred[subgroup_2_mask] == 1) & (y_true[subgroup_2_mask] == 0))

    # Calculate disparities
    accuracy_disparity = np.abs(accuracy_subgroup_1 - accuracy_subgroup_2)
    fpr_disparity = np.abs(fpr_subgroup_1 - fpr_subgroup_2)

    # Calibration
    fraction_of_positives_subgroup_1, mean_predicted_value_subgroup_1 = calibration_curve(y_true[subgroup_1_mask], y_proba[subgroup_1_mask][:, 1], n_bins=10)
    fraction_of_positives_subgroup_2, mean_predicted_value_subgroup_2 = calibration_curve(y_true[subgroup_2_mask], y_proba[subgroup_2_mask][:, 1], n_bins=10)

    calibration_disparity = brier_score_loss(y_true[subgroup_1_mask], y_proba[subgroup_1_mask][:, 1]) - brier_score_loss(y_true[subgroup_2_mask], y_proba[subgroup_2_mask][:, 1])

    print(f"Accuracy Subgroup 1: {accuracy_subgroup_1:}")
    print(f"Accuracy Subgroup 2: {accuracy_subgroup_2:}")
    print(f"Accuracy Disparity: {accuracy_disparity:}")

    print(f"FPR Subgroup 1: {fpr_subgroup_1:}")
    print(f"FPR Subgroup 2: {fpr_subgroup_2:}")
    print(f"FPR Disparity: {fpr_disparity:}")

    print(f"Calibration Disparity: {calibration_disparity:}")
    

# Evaluate model fairness
evaluate_fairness(y_test.values, y_pred, X_test['race'].values, y_proba)

Accuracy Subgroup 1: 0.4761255115961801
Accuracy Subgroup 2: 0.6096579476861167
Accuracy Disparity: 0.1335324360899366
FPR Subgroup 1: 0.0
FPR Subgroup 2: 0.0
FPR Disparity: 0.0
Calibration Disparity: 0.0


Conclusion:

Here's a noticeable difference in accuracy between the two subgroups, suggesting the model performs better for one group over the other, which could indicate bias. Also, the result shows signs of bias in accuracy but performs consistently across groups in terms of calibration. The zero FPR for both groups needs further investigation.